<a href="https://colab.research.google.com/github/41371130H/PL-Repo/blob/main/HW2_%E6%97%A5%E5%B8%B8%E6%94%AF%E5%87%BA%E9%80%9F%E7%AE%97%E8%88%87%E5%88%86%E6%94%A4%2BAI%E5%9B%9E%E9%A5%8B_(%E6%9C%89gradio%E7%89%88).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**在gradio操作完成後，請手動關閉colab儲存格的運行**

In [1]:
# ================================
# ✅ 整合版：Gradio + Gemini 分析 (已修正超時問題)
# ================================

import gspread
import pandas as pd
import datetime
import pytz
import gradio as gr
from google.colab import auth
from google.auth import default
from google.oauth2.service_account import Credentials
import google.generativeai as genai
from google.colab import userdata

# ========== Google Sheets 授權 ==========
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

# 開啟試算表與工作表
SPREADSHEET_URL = "https://docs.google.com/spreadsheets/d/1KhaWfYSh2fGRmwC8SkZG8GdbTshYql4QX-hknk0ON84/edit?usp=sharing"
gsheets = gc.open_by_url(SPREADSHEET_URL)
worksheet1 = gsheets.worksheet("工作表1")  # 記帳資料
worksheet2 = gsheets.worksheet("工作表2")  # 分析結果儲存

# ========== Gemini API ==========
genai.configure(api_key=userdata.get("gemini"))
# 【修改 1】修正模型名稱為官方支援的最新模型
model = genai.GenerativeModel("gemini-2.5-flash")

# ========== 功能 1：新增資料 ==========
def add_record(category, item, amount, aa, people):
    tz = pytz.timezone("Asia/Taipei")
    now = datetime.datetime.now(tz)
    date_str = now.strftime("%Y-%m-%d")
    time_str = now.strftime("%H:%M")

    # 確保 people 是數字且大於 0
    try:
        people = int(people)
        if people <= 0:
            people = 1
    except:
        people = 1

    try:
        amount = float(amount)
    except:
        return "❌ 金額必須是數字", show_records()

    if aa == "yes":
        amount = round(amount / people, 2) # 使用 round 避免小數點過多

    new_data = pd.DataFrame([{
        "日期": date_str,
        "時間": time_str,
        "項目": category,
        "品項": item,
        "金額": amount
    }])

    worksheet1.append_rows(values=new_data.values.tolist(), value_input_option="USER_ENTERED")
    return f"✅ 已新增資料：{category} - {item} - {amount} 元", show_records()

# ========== 功能 2：顯示 / 刪除資料 ==========
def show_records():
    all_data = worksheet1.get_all_values()
    if not all_data or len(all_data) < 2:
        return pd.DataFrame(columns=["列號", "日期", "時間", "項目", "品項", "金額"])
    df = pd.DataFrame(all_data[1:], columns=all_data[0])
    df["金額"] = pd.to_numeric(df["金額"], errors="coerce")
    # 讓「列號」從 0 開始，與 Gradio Dataframe 的索引一致
    return df.reset_index(drop=True).reset_index().rename(columns={"index": "列號"})

def delete_record(row_number):
    try:
        row_to_delete = int(row_number)
        # Google Sheets 的列號是從 1 開始，且標題佔第 1 列
        # 所以 Dataframe 的「列號 0」對應到 Sheet 的第 2 列
        if row_to_delete < 0:
            return "❌ 列號不能小於 0", show_records()

        worksheet1.delete_rows(row_to_delete + 2)
        return f"🗑️ 已刪除第 {row_number} 列資料", show_records()
    except Exception as e:
        return f"❌ 刪除失敗: {e}", show_records()

# ========== 功能 3：Gemini 收支分析 (已修正) ==========
def analyze_records():
    try:
        # 1. 顯示處理中訊息 (優化使用者體驗)
        yield "🔄 正在從 Google Sheets 讀取資料..."
        all_data = worksheet1.get_all_values()
        if not all_data or len(all_data) < 2:
            advice = "⚠️ 沒有資料可以分析"
            return advice

        df = pd.DataFrame(all_data[1:], columns=all_data[0])

        # 2. 準備 Prompt
        prompt = f"""
        以下是我的收支表：
        {df.to_string(index=False)}

        請根據資料，給我收支的總結與三點具體建議，
        包含：花費最多的項目、可能的節省方向、以及理財提醒。
        請使用繁體中文回答。
        """

        # 3. 呼叫 Gemini API
        yield "🧠 Gemini 正在為您分析，請稍候..."
        try:
            # 【修改 2】增加 API 請求的超時時間
            response = model.generate_content(prompt, request_options={"timeout": 120})
            if hasattr(response, 'text') and response.text:
                advice = response.text
                advice = advice.replace("#", "").replace("*", "")
            else:
                advice = "⚠️ Gemini 分析完成，但沒有回傳文字內容。"
        except Exception as e:
            advice = f"❌ Gemini API 呼叫失敗：{e}"

        # 4. 準備寫入 Sheet2
        yield "✍️ 正在將分析結果寫入 Google Sheets..."
        tz = pytz.timezone("Asia/Taipei")
        timestamp = datetime.datetime.now(tz).strftime("%Y-%m-%d %H:%M:%S")

        # 5. 嘗試寫入 Sheet2
        try:
            worksheet2.append_row([timestamp, advice], value_input_option="USER_ENTERED")
        except Exception as e:
            return f"✅ 分析完成，但寫入 Sheet2 失敗：{e}\n\n分析結果：\n{advice}"

        # 6. 回傳最終分析結果到 Gradio
        yield advice

    except Exception as e:
        # 捕獲最外層的錯誤
        return f"❌ 分析流程發生未預期錯誤：{e}"

# ========== Gradio 介面 ==========
with gr.Blocks() as demo:
    gr.Markdown("# 💵 Google Sheet 記帳 + Gemini 分析系統")

    records_table = gr.Dataframe(
        value=show_records(),
        label="📋 目前紀錄",
        interactive=False,
        wrap=True
    )

    with gr.Tab("新增資料"):
        category = gr.Dropdown(["早餐", "午餐", "晚餐", "飲料", "交通", "就學用品", "日常用品", "其他"], label="項目")
        item = gr.Textbox(label="品項")
        amount = gr.Number(label="金額", precision=0)
        aa = gr.Radio(["yes", "no"], label="是否AA分攤", value="no")
        people = gr.Number(label="分攤人數", value=1, precision=0, visible=False)
        add_btn = gr.Button("新增資料")
        add_output = gr.Textbox(label="結果")

        def toggle_people(aa_choice):
            return gr.update(visible=(aa_choice=="yes"), value=2)

        aa.change(fn=toggle_people, inputs=aa, outputs=people)
        add_btn.click(fn=add_record, inputs=[category, item, amount, aa, people], outputs=[add_output, records_table])

    with gr.Tab("刪除資料"):
        row_number = gr.Number(label="要刪除的列號 (請看上方表格的「列號」)", precision=0)
        delete_btn = gr.Button("刪除資料")
        delete_output = gr.Textbox(label="結果")
        delete_btn.click(fn=delete_record, inputs=row_number, outputs=[delete_output, records_table])

    with gr.Tab("Gemini 收支分析"):
        gr.Markdown("點擊按鈕，Gemini 會分析「工作表1」並把結果寫入「工作表2」")
        analyze_btn = gr.Button("開始分析")
        analyze_output = gr.Textbox(label="Gemini 建議", lines=10, interactive=False) # 設定為不可互動

        # 將函式改為生成器 (generator) 的方式，可以即時回報進度
        # Gradio 會自動處理這種情況
        analyze_btn.click(fn=analyze_records, inputs=None, outputs=analyze_output)

# 【修改 3】啟用佇列功能 (Queue)
demo.queue().launch(share=True, debug=True) # 加上 debug=True 可以在 Colab cell 看到更詳細的 log

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://953da6deb5f7ec8f4f.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
